THIS IS TO GET THE INGREDIENTS INTO THE CORRECT FORM

In [1]:
import nltk

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
import nltk

nltk.download('punkt')  
nltk.download('averaged_perceptron_tagger')  
nltk.download('wordnet')  
nltk.download('omw-1.4')  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hijer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hijer\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hijer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\hijer\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [16]:
import re
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag

lemmatizer = WordNetLemmatizer()

def convert_pos_tag_to_wordnet(tag):
    tag = tag.upper()
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('J'):
        return wordnet.ADJ
    return None 

def lemmatize_word(word, tag):
    wordnet_tag = convert_pos_tag_to_wordnet(tag)
    if wordnet_tag:
        return lemmatizer.lemmatize(word, wordnet_tag)
    return word  

fractions = {
    u'\u00BD': '.50',
    u'\u00BC': '.25',
    u'\u00BE': '.75',
    u'\u2153': '.33',
    u'\u2154': '.67',
    u'\u2155': '.20',
    u'\u2156': '.40',
    u'\u2157': '.60',
    u'\u2158': '.80',
    u'\u2159': '.17',
    u'\u215A': '.83',
    u'\u215B': '.13',
    u'\u215C': '.38',
    u'\u215D': '.63',
    u'\u215E': '.88'
}

unwanted_phrases = set([
    "taste", "canned", "is best", "package", "or", "drain", "rinse", "with", "is", "best",
    "to", "cooked", "chopped", "cook", "chop", "not", "evaporate", "cut", "into", "but", "I",
    "prefer", "bite", "sized", "cubes", "and", "cans", "at", "room", "temperature", "pkge",
    "in", "half", "thinly", "diced", "dice", "slice", "sliced", "strip", "strips", "large", "small"
])
unwanted_phrases_pattern = re.compile(r'\b(?:' + '|'.join(map(re.escape, unwanted_phrases)) + r')\b', re.IGNORECASE)
range_pattern = re.compile(r'\(([\d/.\s]+)-([\d/.\s]+)\)')
leading_quantity_pattern = re.compile(r'^\d+\s*\(([^)]+)\)')
standalone_numbers_pattern = re.compile(r'\b\d+(\.\d+)?\b(?!\s*[a-zA-Z])')
punctuation_pattern = re.compile(r'[^\w\s/]')
multiple_spaces_pattern = re.compile(r'\s+')

def convert_fractions(line):
    for key, value in fractions.items():
        if key in line:
            line = line.replace(key, value)
            matches = re.findall(r"\d\s*(?=" + re.escape(value) + ")", line)
            for match in matches:
                number = float(match)
                new_value = float(value)
                multiplier = 1 + new_value
                result = number * multiplier
                string_result = str(result)
                line = line.replace(match + value, string_result)
    return line

def clean_ingredient_text(ingredient):
    # Remove unwanted phrases
    ingredient = unwanted_phrases_pattern.sub('', ingredient)
    
    # Handle ranges like "(1/2 - 1)" by keeping the upper bound
    ingredient = range_pattern.sub(lambda m: m.group(2).strip(), ingredient)
    
    # Keep valid fractions like "1/2" or "3/4"
    ingredient = re.sub(r'\b(\d+/\d+)', r'\1', ingredient)
    
    # Remove leading quantities in parentheses, e.g., "2 (21 ounce)" -> "21 ounce"
    ingredient = leading_quantity_pattern.sub(r'\1', ingredient)
    
    # Remove any standalone numbers not tied to units
    ingredient = standalone_numbers_pattern.sub('', ingredient)
    
    # Remove punctuation but keep slashes (for fractions) and collapse multiple spaces
    ingredient = punctuation_pattern.sub('', ingredient)
    ingredient = multiple_spaces_pattern.sub(' ', ingredient).strip()

    return ingredient

def process_ingredient(ingredient):
    ingredient = convert_fractions(ingredient)
    cleaned_ingredient = clean_ingredient_text(ingredient)
    tokens = word_tokenize(cleaned_ingredient)
    tagged_tokens = pos_tag(tokens)
    lemmatized_tokens = [lemmatize_word(word, tag) for word, tag in tagged_tokens]
    
    return " ".join(lemmatized_tokens)

def process_ingredient_list(ingredient_list):
    ingredients = [ingredient.strip() for ingredient in ingredient_list.split(',')]
    processed_ingredients = [process_ingredient(ingredient) for ingredient in ingredients]
    
    return ', '.join(processed_ingredients)

# Test the function with an example list
example_ingredients_list = "3/4 cup unsalted butter, 1/2 cup sugar, 1/2 cup"
processed_ingredients_list = process_ingredient_list(example_ingredients_list)
print(processed_ingredients_list)  # Output: "3/4 cup unsalted butter, 1/2 cup sugar, 1/2 cup"






/4 cup unsalted butter, /2 cup sugar, /2 cup


In [ ]:
#Best verion can do it all except handle ranges
import re
from fractions import Fraction
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
import nltk

lemmatizer = WordNetLemmatizer()

def convert_pos_tag_to_wordnet(tag):
    tag = tag.upper()
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('J'):
        return wordnet.ADJ
    return None 

def lemmatize_word(word, tag):
    wordnet_tag = convert_pos_tag_to_wordnet(tag)
    if wordnet_tag:
        return lemmatizer.lemmatize(word, wordnet_tag)
    return word  

def convert_fractions_to_decimals(text):
    def fraction_to_decimal(match):
        try:
            fraction = match.group(0)
            return str(round(float(Fraction(fraction)), 2))
        except ValueError:
            return fraction  # Return the original if conversion fails

    return re.sub(r'\b\d+/\d+\b', fraction_to_decimal, text)

def clean_ingredient_text(ingredient):
    # List of unwanted phrases to remove
    unwanted_phrases = [
        "taste", "canned", "is best", "package", "or", "drain", "rinse", "with", "is", "best",
        "to", "cooked", "chopped", "cook", "chop", "not", "evaporate", "cut", "into", "but", "I",
        "prefer", "bite", "sized", "cubes", "and", "cans", "at", "room", "temperature", "pkge",
        "in", "half", "thinly", "diced", "dice", "slice", "sliced", "strip", "strips", "large", "small"
    ]
    
    # Remove unwanted phrases
    ingredient = re.sub(r'\b(?:' + '|'.join(unwanted_phrases) + r')\b', '', ingredient, flags=re.IGNORECASE)
    
    # Handle ranges like "(1/2 - 1)" by keeping the upper bound
    ingredient = re.sub(r'\(([\d/.\s]+)-([\d/.\s]+)\)', lambda m: m.group(2).strip(), ingredient)
    
    # Convert fractions to decimals
    ingredient = convert_fractions_to_decimals(ingredient)
    
    # Remove leading quantities in parentheses, e.g., "2 (21 ounce)" -> "21 ounce"
    ingredient = re.sub(r'\d+\s*\(([^)]+)\)', r'\1', ingredient)
    
    # Remove punctuation but keep slashes (for fractions) and collapse multiple spaces
    ingredient = re.sub(r'[^\w\s/.]', '', ingredient)
    ingredient = re.sub(r'\s+', ' ', ingredient).strip()

    return ingredient

def process_ingredient(ingredient):
    cleaned_ingredient = clean_ingredient_text(ingredient)
    tokens = word_tokenize(cleaned_ingredient)
    tagged_tokens = pos_tag(tokens)
    lemmatized_tokens = [lemmatize_word(word, tag) for word, tag in tagged_tokens]
    
    return " ".join(lemmatized_tokens)

def process_ingredient_list(ingredient_list):
    ingredients = [ingredient.strip() for ingredient in ingredient_list.split(',')]
    processed_ingredients = [process_ingredient(ingredient) for ingredient in ingredients]
    
    return ', '.join(processed_ingredients)

# Test the function with an example list
example_ingredients_list = "2 (21 ounce) cherry pie filling, 2 egg, 1 (14 ounce), (1/2 - 1) lb corn beef"
processed_ingredients_list = process_ingredient_list(example_ingredients_list)
print(processed_ingredients_list)  # Output: "21 ounce cherry pie filling, 2 egg, 14 ounce, 1 lb corn beef"

21 ounce cherry pie filling, 2 egg, 14 ounce, 1 lb corn beef


In [1]:
import re
from fractions import Fraction
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag

lemmatizer = WordNetLemmatizer()

def convert_pos_tag_to_wordnet(tag):
    tag = tag.upper()
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('J'):
        return wordnet.ADJ
    return None 

def lemmatize_word(word, tag):
    wordnet_tag = convert_pos_tag_to_wordnet(tag)
    if wordnet_tag:
        return lemmatizer.lemmatize(word, wordnet_tag)
    return word  

def convert_fractions_to_decimals(text):
    """
    Converts fractions in the text to decimals.
    Example: '1/2' -> '0.5', '3/4' -> '0.75'
    """
    def fraction_to_decimal(match):
        try:
            fraction = match.group(0)
            return str(round(float(Fraction(fraction)), 2))
        except ValueError:
            return fraction  # Return the original if conversion fails

    return re.sub(r'\b\d+/\d+\b', fraction_to_decimal, text)

def handle_ranges(text):
    """
    Handles ranges like '(1/2 - 1)' or '(0.5 - 1)' and keeps the maximum value.
    """
    def extract_max(match):
        try:
            # Extract both parts of the range
            lower, upper = match.groups()
            # Convert to float (handle fractions if necessary)
            lower_value = float(Fraction(lower)) if '/' in lower else float(lower)
            upper_value = float(Fraction(upper)) if '/' in upper else float(upper)
            # Return the maximum value
            return str(round(max(lower_value, upper_value), 2))
        except ValueError:
            return match.group(0)  # Return the original if conversion fails

    return re.sub(r'\(([\d/.\s]+)-([\d/.\s]+)\)', extract_max, text)

def clean_ingredient_text(ingredient):
    # List of unwanted phrases to remove
    unwanted_phrases = [
        "taste", "canned", "is best", "package", "or", "drain", "rinse", "with", "is", "best",
        "to", "cooked", "chopped", "cook", "chop", "not", "evaporate", "cut", "into", "but", "I",
        "prefer", "bite", "sized", "cubes", "and", "cans", "at", "room", "temperature", "pkge",
        "in", "half", "thinly", "diced", "dice", "slice", "sliced", "strip", "strips", "large", "small"
    ]
    
    # Remove unwanted phrases
    ingredient = re.sub(r'\b(?:' + '|'.join(unwanted_phrases) + r')\b', '', ingredient, flags=re.IGNORECASE)
    
    # Handle ranges by keeping the maximum value
    ingredient = handle_ranges(ingredient)
    
    # Convert fractions to decimals
    ingredient = convert_fractions_to_decimals(ingredient)
    
    # Remove leading quantities in parentheses, e.g., "2 (21 ounce)" -> "21 ounce"
    ingredient = re.sub(r'\d+\s*\(([^)]+)\)', r'\1', ingredient)
    
    # Remove punctuation but keep slashes (for fractions) and collapse multiple spaces
    ingredient = re.sub(r'[^\w\s/.]', '', ingredient)
    ingredient = re.sub(r'\s+', ' ', ingredient).strip()

    return ingredient

def process_ingredient(ingredient):
    cleaned_ingredient = clean_ingredient_text(ingredient)
    tokens = word_tokenize(cleaned_ingredient)
    tagged_tokens = pos_tag(tokens)
    lemmatized_tokens = [lemmatize_word(word, tag) for word, tag in tagged_tokens]
    
    return " ".join(lemmatized_tokens)

def process_ingredient_list(ingredient_list):
    ingredients = [ingredient.strip() for ingredient in ingredient_list.split(',')]
    processed_ingredients = [process_ingredient(ingredient) for ingredient in ingredients]
    
    return ', '.join(processed_ingredients)

# Test the function with an example list
example_ingredients_list = "2 (21 ounce) cherry pie filling, 2 egg, 1 (14 ounce), (1/2 - 1) lb corn beef, (3/4 - 2) cup milk"
processed_ingredients_list = process_ingredient_list(example_ingredients_list)
print(processed_ingredients_list)

21 ounce cherry pie filling, 2 egg, 14 ounce, 1.0 lb corn beef, 2.0 cup milk


In [ ]:
#CURRENT BEST 11/27 1:40
import re
from fractions import Fraction
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
import nltk

lemmatizer = WordNetLemmatizer()

def convert_pos_tag_to_wordnet(tag):
    tag = tag.upper()
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('J'):
        return wordnet.ADJ
    return None 

def lemmatize_word(word, tag):
    wordnet_tag = convert_pos_tag_to_wordnet(tag)
    if wordnet_tag:
        return lemmatizer.lemmatize(word, wordnet_tag)
    return word  

def convert_fractions_to_decimals(text):
    """
    Converts fractions in the text to decimals.
    Example: '1/2' -> '0.5', '3/4' -> '0.75'
    """
    def fraction_to_decimal(match):
        try:
            fraction = match.group(0)
            return str(round(float(Fraction(fraction)), 2))
        except ValueError:
            return fraction  # Return the original if conversion fails

    return re.sub(r'\b\d+/\d+\b', fraction_to_decimal, text)

def clean_ingredient_text(ingredient):
    # List of unwanted phrases to remove
    unwanted_phrases = [
        "taste", "canned", "is best", "package", "or", "drain", "rinse", "with", "is", "best",
        "to", "cooked", "chopped", "cook", "chop", "not", "evaporate", "cut", "into", "but", "I",
        "prefer", "bite", "sized", "cubes", "and", "cans", "at", "room", "temperature", "pkge",
        "in", "half", "thinly", "diced", "dice", "slice", "sliced", "strip", "strips", "large", "small"
    ]
    
    # Remove unwanted phrases
    ingredient = re.sub(r'\b(?:' + '|'.join(unwanted_phrases) + r')\b', '', ingredient, flags=re.IGNORECASE)
    
    # Handle ranges like "(1/2 - 1)" by keeping the upper bound
    ingredient = re.sub(r'\(([\d/.\s]+)-([\d/.\s]+)\)', lambda m: str(max(float(Fraction(m.group(1).strip())), float(Fraction(m.group(2).strip())))), ingredient)
    
    # Convert fractions to decimals
    ingredient = convert_fractions_to_decimals(ingredient)
    
    # Remove leading quantities in parentheses, e.g., "2 (21 ounce)" -> "21 ounce"
    ingredient = re.sub(r'\d+\s*\(([^)]+)\)', r'\1', ingredient)
    
    # Remove punctuation but keep slashes (for fractions) and collapse multiple spaces
    ingredient = re.sub(r'[^\w\s/.]', '', ingredient)
    ingredient = re.sub(r'\s+', ' ', ingredient).strip()

    return ingredient

def process_ingredient(ingredient):
    cleaned_ingredient = clean_ingredient_text(ingredient)
    tokens = word_tokenize(cleaned_ingredient)
    tagged_tokens = pos_tag(tokens)
    lemmatized_tokens = [lemmatize_word(word, tag) for word, tag in tagged_tokens]
    
    return " ".join(lemmatized_tokens)

def process_ingredient_list(ingredient_list):
    ingredients = [ingredient.strip() for ingredient in ingredient_list.split(',')]
    processed_ingredients = [process_ingredient(ingredient) for ingredient in ingredients]
    
    return ', '.join(processed_ingredients)

# Test the function with an example list
example_ingredients_list = "2 (21 ounce) cherry pie filling, 2 egg, 1 (14 ounce), (1/2 - 1) lb corn beef"
processed_ingredients_list = process_ingredient_list(example_ingredients_list)
print(processed_ingredients_list)  # Output: "21 ounce cherry pie filling, 2 egg, 14 ounce, 1 lb corn beef"

21 ounce cherry pie filling, 2 egg, 14 ounce, 1.0 lb corn beef


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hijer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hijer\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hijer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import pandas as pd

# Read documentation!, use ast literal eval
# Try catch with literal eval, converters
df = pd.read_csv('Cleaned_Ingredients/recipes_food_com_cleaned.csv')
df = df.drop(28217)
df = df.drop(84485)
df = df.drop(139400)
df = df.drop(169196)
df['NLP_Ingredients'] = df['IngredientsRaw'].apply(process_ingredient_list)
df = df[['ID', 'Name','NLP_Ingredients', 'Cleaned_Ingredients']]
df.head(10)

In [ ]:
#current best
import re
from fractions import Fraction
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
import nltk

lemmatizer = WordNetLemmatizer()

def convert_pos_tag_to_wordnet(tag):
    tag = tag.upper()
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('J'):
        return wordnet.ADJ
    return None 

def lemmatize_word(word, tag):
    wordnet_tag = convert_pos_tag_to_wordnet(tag)
    if wordnet_tag:
        return lemmatizer.lemmatize(word, wordnet_tag)
    return word  

def convert_fractions_to_decimals(text):
    """
    Converts fractions in the text to decimals and multiplies them with the preceding number if present.
    Example: '1/2' -> '0.5', '3/4' -> '0.75', '2 1/4' -> '0.5'
    """
    def fraction_to_decimal(match):
        try:
            whole_number = match.group(1)
            fraction = match.group(2)
            if whole_number:
                return str(round(float(whole_number) * float(Fraction(fraction)), 2))
            else:
                return str(round(float(Fraction(fraction)), 2))
        except ValueError:
            return match.group(0)  # Return the original if conversion fails

    return re.sub(r'(\d+)\s+(\d+/\d+)', fraction_to_decimal, text)

def clean_ingredient_text(ingredient):
    # List of unwanted phrases to remove
    unwanted_phrases = [
        "taste", "canned", "is best", "package", "or", "drain", "rinse", "with", "is", "best",
        "to", "cooked", "chopped", "cook", "chop", "not", "evaporate", "cut", "into", "but", "I",
        "prefer", "bite", "sized", "cubes", "and", "cans", "at", "room", "temperature", "pkge",
        "in", "half", "thinly", "diced", "dice", "slice", "sliced", "strip", "strips", "large", "small"
    ]
    
    # Remove unwanted phrases
    ingredient = re.sub(r'\b(?:' + '|'.join(unwanted_phrases) + r')\b', '', ingredient, flags=re.IGNORECASE)
    
    # Handle ranges like "(1/2 - 1)" by keeping the upper bound
    ingredient = re.sub(r'\(([\d/.\s]+)-([\d/.\s]+)\)', lambda m: str(max(float(Fraction(m.group(1).strip())), float(Fraction(m.group(2).strip())))), ingredient)
    
    # Convert fractions to decimals and multiply with preceding number if present
    ingredient = convert_fractions_to_decimals(ingredient)
    
    # Convert standalone fractions to decimals
    ingredient = re.sub(r'\b(\d+/\d+)\b', lambda m: str(round(float(Fraction(m.group(1))), 2)), ingredient)
    
    # Remove leading quantities in parentheses, e.g., "2 (21 ounce)" -> "21 ounce"
    ingredient = re.sub(r'\d+\s*\(([^)]+)\)', r'\1', ingredient)
    
    # Remove punctuation but keep slashes (for fractions) and collapse multiple spaces
    ingredient = re.sub(r'[^\w\s/.]', '', ingredient)
    ingredient = re.sub(r'\s+', ' ', ingredient).strip()

    return ingredient

def process_ingredient(ingredient):
    cleaned_ingredient = clean_ingredient_text(ingredient)
    tokens = word_tokenize(cleaned_ingredient)
    tagged_tokens = pos_tag(tokens)
    lemmatized_tokens = [lemmatize_word(word, tag) for word, tag in tagged_tokens]
    
    return " ".join(lemmatized_tokens)

def process_ingredient_list(ingredient_list):
    ingredients = [ingredient.strip() for ingredient in ingredient_list.split(',')]
    processed_ingredients = [process_ingredient(ingredient) for ingredient in ingredients]
    
    return ', '.join(processed_ingredients)

# Test the function with an example list
example_ingredients_list = "2 (21 ounce) cherry pie filling, 2 egg, 1 (14 ounce), (1/2 - 1) lb corn beef, 2 1/4 cup biscuit mix"
processed_ingredients_list = process_ingredient_list(example_ingredients_list)
print(processed_ingredients_list)  # Output: "21 ounce cherry pie filling, 2 egg, 14 ounce, 1 lb corn beef, 0.5 cup biscuit mix"

21 ounce cherry pie filling, 2 egg, 14 ounce, 1.0 lb corn beef, 0.5 cup biscuit mix


In [ ]:
import re
from fractions import Fraction
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
import nltk

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def convert_pos_tag_to_wordnet(tag):
    tag = tag.upper()
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('J'):
        return wordnet.ADJ
    return None 

def lemmatize_word(word, tag):
    wordnet_tag = convert_pos_tag_to_wordnet(tag)
    if wordnet_tag:
        return lemmatizer.lemmatize(word, wordnet_tag)
    return word  

def convert_fractions_to_decimals(text):
    """
    Converts fractions in the text to decimals and multiplies them with the preceding number if present.
    Example: '1/2' -> '0.5', '3/4' -> '0.75', '2 1/4' -> '0.5'
    """
    def fraction_to_decimal(match):
        try:
            whole_number = match.group(1)
            fraction = match.group(2)
            if whole_number:
                return str(round(float(whole_number) * float(Fraction(fraction)), 2))
            else:
                return str(round(float(Fraction(fraction)), 2))
        except ValueError:
            return match.group(0)  # Return the original if conversion fails

    return re.sub(r'(\d+)\s+(\d+/\d+)', fraction_to_decimal, text)

def clean_ingredient_text(ingredient):
    # List of unwanted phrases to remove
    unwanted_phrases = [
        "taste", "canned", "is best", "package", "or", "drain", "rinse", "with", "is", "best",
        "to", "cooked", "chopped", "cook", "chop", "not", "evaporate", "cut", "into", "but", "I",
        "prefer", "bite", "sized", "cubes", "and", "cans", "at", "room", "temperature", "pkge",
        "in", "half", "thinly", "diced", "dice", "slice", "sliced", "strip", "strips", "large", "small"
    ]
    
    # Remove unwanted phrases
    ingredient = re.sub(r'\b(?:' + '|'.join(unwanted_phrases) + r')\b', '', ingredient, flags=re.IGNORECASE)
    
    # Handle ranges like "(1/2 - 1)" by keeping the upper bound
    ingredient = re.sub(r'\(([\d/.\s]+)-([\d/.\s]+)\)', lambda m: str(max(float(Fraction(m.group(1).strip())), float(Fraction(m.group(2).strip())))), ingredient)
    
    # Convert fractions to decimals and multiply with preceding number if present
    ingredient = convert_fractions_to_decimals(ingredient)
    
    # Convert standalone fractions to decimals
    ingredient = re.sub(r'\b(\d+/\d+)\b', lambda m: str(round(float(Fraction(m.group(1))), 2)), ingredient)
    
    # Remove leading quantities in parentheses, e.g., "2 (21 ounce)" -> "21 ounce"
    ingredient = re.sub(r'\d+\s*\(([^)]+)\)', r'\1', ingredient)
    
    # Remove punctuation but keep slashes (for fractions) and collapse multiple spaces
    ingredient = re.sub(r'[^\w\s/.]', '', ingredient)
    ingredient = re.sub(r'\s+', ' ', ingredient).strip()

    return ingredient

def process_ingredient(ingredient):
    cleaned_ingredient = clean_ingredient_text(ingredient)
    tokens = word_tokenize(cleaned_ingredient)
    tagged_tokens = pos_tag(tokens)
    lemmatized_tokens = [lemmatize_word(word, tag) for word, tag in tagged_tokens]
    
    return " ".join(lemmatized_tokens)

def process_ingredient_list(ingredient_list):
    ingredients = [ingredient.strip() for ingredient in ingredient_list.split(',')]
    processed_ingredients = [process_ingredient(ingredient) for ingredient in ingredients]
    
    return ', '.join(processed_ingredients)

# Test the function with an example list
example_ingredients_list = "2 (21 ounce) cherry pie filling, 2 egg, 1 (14 ounce), (1/2 - 1) lb corn beef, 2 1/4 cup biscuit mix"
processed_ingredients_list = process_ingredient_list(example_ingredients_list)
print(processed_ingredients_list)  # Output: "21 ounce cherry pie filling, 2 egg, 14 ounce, 1 lb corn beef, 0.5 cup biscuit mix"

21 ounce cherry pie filling, 2 egg, 14 ounce, 1.0 lb corn beef, 0.5 cup biscuit mix


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hijer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hijer\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hijer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
#Debugging 

In [11]:
invalid_entries = df[df['IngredientsRaw'] == '.']
print(invalid_entries)
empty_rows = df[df.isnull().any(axis=1)]
empty_rows

Empty DataFrame
Columns: [Unnamed: 0.1, ID, Name, Description, IngredientsExtracted, IngredientsRaw, Instructions, Servings, TotalTime, Calories, FatContent, SaturatedFatContent, CholesterolContent, SodiumContent, CarbohydrateContent, FiberContent, SugarContent, ProteinContent, Unnamed: 0, RecipeId, Raw_Ingredients, Cleaned_Ingredients, IngredientsRawInGrams]
Index: []

[0 rows x 23 columns]


,Unnamed: 0.1,ID,Name,Description,IngredientsExtracted,IngredientsRaw,Instructions,Servings,TotalTime,Calories,...,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,Unnamed: 0,RecipeId,Raw_Ingredients,Cleaned_Ingredients,IngredientsRawInGrams
28217,28672,537794,Cheesy Fried Chicken Parm Balls,A simple homemade chicken nugget is stuffed wi...,mozzarella string cheese ground chicken italia...,"[""2 mozzarella string cheese"",""1 lb ground chi...","[""Cut each mozzarella stick in half lengthwise...",NaN,50,151.7,...,517.6,16.8,0.8,0.8,10.0,28880,537794,"['salt', 'grated parmesan cheese', 'ground chi...","['salt', 'grated parmesan cheese', 'pepper', '...",907.184
84485,85741,537778,Steamed Pork & Scallion Dumplings,These dumplings are stuffed with a savory pork...,low sodium soy sauce rice vinegar chili oil sc...,"[""1/2 cup low sodium soy sauce"",""2 tablespoons...","[""Whisk together the ingredients for the dippi...",NaN,70,68.9,...,224.9,7.0,0.5,1.1,3.4,86535,537778,"['low sodium soy sauce', 'garlic cloves', 'sca...","['low sodium soy sauce', 'garlic cloves', 'sca...",226.796
139400,141366,537785,Raspberry Almond Shortbread Cookies,A wonderful recipe from Better Homes Garden. T...,butter granulated sugar almond extract seedles...,"[""1 cup butter, softened"",""2/3 cup granulated ...","[""In a medium bowl beat butter medium speed fo...",NaN,40,113.5,...,46.7,16.1,0.2,9.9,0.8,142848,537785,"['granulated sugar', 'sugar', 'all-purpose flo...","['granulated sugar', 'sugar', 'all-purpose flo...",240.0
169196,171452,537795,Chicken Parm Biscuits,Buttermilk biscuits are filled with fried chic...,panko breadcrumbs grated parmesan cheese garli...,"[""1 1/2 cups panko breadcrumbs"",""1/3 cup grate...","[""Preheat oven to 400 F and lightly grease a b...",NaN,60,360.7,...,1204.2,41.0,1.5,4.8,10.1,173399,537795,"['mozzarella cheese', 'salt', 'grated parmesan...","['mozzarella cheese', 'salt', 'grated parmesan...",360.0


In [1]:
import re
from fractions import Fraction
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
import nltk

lemmatizer = WordNetLemmatizer()

def convert_pos_tag_to_wordnet(tag):
    tag = tag.upper()
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('J'):
        return wordnet.ADJ
    return None 

def lemmatize_word(word, tag):
    wordnet_tag = convert_pos_tag_to_wordnet(tag)
    if wordnet_tag:
        return lemmatizer.lemmatize(word, wordnet_tag)
    return word  

def convert_fractions_to_decimals(text):
    """
    Converts fractions in the text to decimals and multiplies them with the preceding number if present.
    Example: '1/2' -> '0.5', '3/4' -> '0.75', '2 1/4' -> '0.5'
    """
    def fraction_to_decimal(match):
        try:
            whole_number = match.group(1)
            fraction = match.group(2)
            print(f"Debug: whole_number = {whole_number}, fraction = {fraction}")  # Debugging here
            if whole_number:
                return str(round(float(whole_number) * float(Fraction(fraction)), 2))
            else:
                return str(round(float(Fraction(fraction)), 2))
        except ValueError as e:
            print(f"Error in fraction_to_decimal: {e}, match = {match.group(0)}")  # Debugging error
            return match.group(0)  # Return the original if conversion fails

    return re.sub(r'(\d+)\s+(\d+/\d+)', fraction_to_decimal, text)

def clean_ingredient_text(ingredient):
    # List of unwanted phrases to remove
    unwanted_phrases = [
        "taste", "canned", "is best", "package", "or", "drain", "rinse", "with", "is", "best",
        "to", "cooked", "chopped", "cook", "chop", "not", "evaporate", "cut", "into", "but", "I",
        "prefer", "bite", "sized", "cubes", "and", "cans", "at", "room", "temperature", "pkge",
        "in", "half", "thinly", "diced", "dice", "slice", "sliced", "strip", "strips", "large", "small"
    ]
    
    print(f"Original ingredient: {ingredient}")  # Debug start

    # Remove unwanted phrases
    ingredient = re.sub(r'\b(?:' + '|'.join(unwanted_phrases) + r')\b', '', ingredient, flags=re.IGNORECASE)
    print(f"After removing unwanted phrases: {ingredient}")  # Debug step

    # Handle ranges like "(1/2 - 1)" by keeping the upper bound
    ingredient = re.sub(r'\(([\d/.\s]+)-([\d/.\s]+)\)', 
                        lambda m: str(max(float(Fraction(m.group(1).strip())), float(Fraction(m.group(2).strip())))), 
                        ingredient)
    print(f"After handling ranges: {ingredient}")  # Debug step

    # Convert fractions to decimals and multiply with preceding number if present
    ingredient = convert_fractions_to_decimals(ingredient)
    print(f"After converting fractions to decimals: {ingredient}")  # Debug step

    # Convert standalone fractions to decimals
    ingredient = re.sub(r'\b(\d+/\d+)\b', lambda m: str(round(float(Fraction(m.group(1))), 2)), ingredient)
    print(f"After removing standalone fractions: {ingredient}")  # Debug step

    # Remove leading quantities in parentheses, e.g., "2 (21 ounce)" -> "21 ounce"
    ingredient = re.sub(r'\d+\s*\(([^)]+)\)', r'\1', ingredient)
    print(f"After removing leading quantities: {ingredient}")  # Debug step

    # Remove punctuation but keep slashes (for fractions) and collapse multiple spaces
    ingredient = re.sub(r'[^\w\s/.]', '', ingredient)
    ingredient = re.sub(r'\s+', ' ', ingredient).strip()
    print(f"Final cleaned ingredient: {ingredient}")  # Debug final output

    return ingredient

def process_ingredient(ingredient):
    cleaned_ingredient = clean_ingredient_text(ingredient)
    print(f"Cleaned ingredient: {cleaned_ingredient}")  # Debug cleaned output

    tokens = word_tokenize(cleaned_ingredient)
    print(f"Tokens: {tokens}")  # Debug tokens

    tagged_tokens = pos_tag(tokens)
    print(f"Tagged Tokens: {tagged_tokens}")  # Debug tagging

    lemmatized_tokens = [lemmatize_word(word, tag) for word, tag in tagged_tokens]
    print(f"Lemmatized Tokens: {lemmatized_tokens}")  # Debug lemmatization

    return " ".join(lemmatized_tokens)

def process_ingredient_list(ingredient_list):
    print(f"Original ingredient list: {ingredient_list}")  # Debug input

    ingredients = [ingredient.strip() for ingredient in ingredient_list.split(',')]
    print(f"Split ingredients: {ingredients}")  # Debug splitting

    processed_ingredients = []
    for ingredient in ingredients:
        try:
            processed = process_ingredient(ingredient)
            processed_ingredients.append(processed)
        except Exception as e:
            print(f"Error processing ingredient: {ingredient}, Error: {e}")  # Debug error

    print(f"Processed ingredients: {processed_ingredients}")  # Debug final output
    return ', '.join(processed_ingredients)

# Test the function with an example list
example_ingredients_list = "2 (21 ounce) cherry pie filling, 2 egg, 1 (14 ounce), (1/2 - 1) lb corn beef, 2 1/4 cup biscuit mix"
processed_ingredients_list = process_ingredient_list(example_ingredients_list)
print(processed_ingredients_list)  # Output: Processed result with debug info


Original ingredient list: 2 (21 ounce) cherry pie filling, 2 egg, 1 (14 ounce), (1/2 - 1) lb corn beef, 2 1/4 cup biscuit mix
Split ingredients: ['2 (21 ounce) cherry pie filling', '2 egg', '1 (14 ounce)', '(1/2 - 1) lb corn beef', '2 1/4 cup biscuit mix']
Original ingredient: 2 (21 ounce) cherry pie filling
After removing unwanted phrases: 2 (21 ounce) cherry pie filling
After handling ranges: 2 (21 ounce) cherry pie filling
After converting fractions to decimals: 2 (21 ounce) cherry pie filling
After removing standalone fractions: 2 (21 ounce) cherry pie filling
After removing leading quantities: 21 ounce cherry pie filling
Final cleaned ingredient: 21 ounce cherry pie filling
Cleaned ingredient: 21 ounce cherry pie filling
Tokens: ['21', 'ounce', 'cherry', 'pie', 'filling']
Tagged Tokens: [('21', 'CD'), ('ounce', 'NN'), ('cherry', 'NN'), ('pie', 'NN'), ('filling', 'NN')]
Lemmatized Tokens: ['21', 'ounce', 'cherry', 'pie', 'filling']
Original ingredient: 2 egg
After removing unwanted